In [1]:
# Improved Restaurant RAG Chatbot Implementation
# For Google Colab with T4 GPU

# Install required packages
!pip install -q langchain langchain_community sentence_transformers faiss-cpu chromadb gradio transformers accelerate

# Import necessary libraries
import pandas as pd
import numpy as np
import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain.schema import format_document
from langchain_core.documents import Document
import gradio as gr

print("Loading and preprocessing data...")

# 1. Load and Preprocess Data
restaurant_info = pd.read_csv('/content/merged_restaurants (5).csv')
menu_items = pd.read_csv('/content/FINAL_BY_ME_Merged.csv')

# Clean and preprocess data
restaurant_info = restaurant_info.fillna('N/A')
menu_items = menu_items.fillna('N/A')

print("Creating enhanced knowledge base documents...")

# 2. Create Enhanced Knowledge Base Documents
documents = []

# Create more structured documents from restaurant information
for idx, row in restaurant_info.iterrows():
    content = f"""RESTAURANT INFORMATION:
Name: {row['name']}
Location: {row['location']}
Phone: {row['phone']}
Email: {row['email']}
Operating Hours:
  - Breakfast: {row['breakfast']}
  - Lunch: {row['lunch']}
  - Dinner: {row['dinner']}
Cuisine Type: {row['cuisine']}
"""
    documents.append(Document(page_content=content, metadata={"source": "restaurant_info", "name": row['name'], "type": "information"}))

# More detailed processing of menu items with better structure
# Process by restaurant
for restaurant in menu_items['restaurant'].unique():
    # Create a comprehensive menu document for each restaurant
    full_menu_content = f"FULL MENU FOR {restaurant}:\n\n"

    restaurant_menu = menu_items[menu_items['restaurant'] == restaurant]

    # Group by sections
    for section in restaurant_menu['section'].unique():
        if pd.notna(section) and section != "":
            section_items = restaurant_menu[restaurant_menu['section'] == section]
            full_menu_content += f"SECTION: {section}\n"

            for _, item in section_items.iterrows():
                price = item['price'] if pd.notna(item['price']) else "N/A"
                description = item['description'] if pd.notna(item['description']) else "No description available"
                full_menu_content += f"- Item: {item['item']}, Price: {price}\n  Description: {description}\n"

            full_menu_content += "\n"

    documents.append(Document(page_content=full_menu_content, metadata={"source": "menu", "restaurant": restaurant, "type": "full_menu"}))

    # Create specialized documents for dietary options
    vegetarian_items = []
    dessert_items = []
    beverage_items = []

    for _, item in restaurant_menu.iterrows():
        item_name = str(item['item']).lower()
        description = str(item['description']).lower() if pd.notna(item['description']) else ""
        price = item['price'] if pd.notna(item['price']) else "N/A"

        # Check for vegetarian items
        veg_keywords = ['vegetarian', 'paneer', 'cheese', 'palak', 'spinach', 'salad', 'cottage cheese', 'veg', 'greek salad']
        if any(keyword in item_name.lower() for keyword in veg_keywords) or any(keyword in description for keyword in veg_keywords):
            vegetarian_items.append(f"- {item['item']}, Price: {price}, Description: {item['description']}")

        # Check for dessert items
        dessert_keywords = ['cake', 'pastry', 'sweet', 'chocolate', 'truffle', 'velvet', 'cheesecake', 'halwa', 'dessert']
        if any(keyword in item_name.lower() for keyword in dessert_keywords) or any(keyword in description for keyword in dessert_keywords):
            dessert_items.append(f"- {item['item']}, Price: {price}, Description: {item['description']}")

        # Check for beverages
        beverage_keywords = ['tea', 'coffee', 'juice', 'water', 'cola', 'beer', 'wine', 'cocktail', 'red bull', 'aerated', 'iced']
        if any(keyword in item_name.lower() for keyword in beverage_keywords) or any(keyword in description for keyword in beverage_keywords):
            beverage_items.append(f"- {item['item']}, Price: {price}, Description: {item['description']}")

    # Add specialized dietary documents
    if vegetarian_items:
        veg_content = f"VEGETARIAN OPTIONS AT {restaurant}:\n" + "\n".join(vegetarian_items)
        documents.append(Document(page_content=veg_content, metadata={"source": "menu", "restaurant": restaurant, "type": "vegetarian"}))

    if dessert_items:
        dessert_content = f"DESSERT OPTIONS AT {restaurant}:\n" + "\n".join(dessert_items)
        documents.append(Document(page_content=dessert_content, metadata={"source": "menu", "restaurant": restaurant, "type": "dessert"}))

    if beverage_items:
        beverage_content = f"BEVERAGE OPTIONS AT {restaurant}:\n" + "\n".join(beverage_items)
        documents.append(Document(page_content=beverage_content, metadata={"source": "menu", "restaurant": restaurant, "type": "beverage"}))

# Add comparative documents
# Compare Avartana locations
avartana_locations = restaurant_info[restaurant_info['name'] == 'Avartana']
if not avartana_locations.empty:
    avartana_comparison = "COMPARISON OF AVARTANA LOCATIONS:\n\n"

    for idx, row in avartana_locations.iterrows():
        avartana_comparison += f"Location: {row['location']}\n"
        avartana_comparison += f"Hours: Breakfast - {row['breakfast']}, Lunch - {row['lunch']}, Dinner - {row['dinner']}\n"
        avartana_comparison += f"Cuisine: {row['cuisine']}\n\n"

    documents.append(Document(page_content=avartana_comparison, metadata={"source": "comparison", "restaurant": "Avartana", "type": "location_comparison"}))

# Add price range documents
for restaurant in menu_items['restaurant'].unique():
    restaurant_menu = menu_items[menu_items['restaurant'] == restaurant]

    # Process prices
    prices = []
    for _, item in restaurant_menu.iterrows():
        if pd.notna(item['price']) and isinstance(item['price'], str):
            # Extract numeric part of the price
            price_match = re.search(r'₹?(\d+)', str(item['price']))
            if price_match:
                try:
                    prices.append(int(price_match.group(1)))
                except:
                    pass

    if prices:
        min_price = min(prices)
        max_price = max(prices)
        avg_price = sum(prices) / len(prices)

        price_content = f"""PRICE RANGE FOR {restaurant}:
Minimum Price: ₹{min_price}
Maximum Price: ₹{max_price}
Average Price: ₹{int(avg_price)}

Price ranges by category (in ₹):
- Budget (under ₹500): {sum(1 for p in prices if p < 500)} items
- Mid-range (₹500-₹1000): {sum(1 for p in prices if 500 <= p <= 1000)} items
- Premium (₹1000-₹1500): {sum(1 for p in prices if 1000 < p <= 1500)} items
- Luxury (above ₹1500): {sum(1 for p in prices if p > 1500)} items
"""
        documents.append(Document(page_content=price_content, metadata={"source": "analysis", "restaurant": restaurant, "type": "price_range"}))

print(f"Created {len(documents)} enhanced knowledge base documents")

print("Splitting documents...")

# 3. Split Documents with smaller chunk size for more precise retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Smaller chunks for more precise retrieval
    chunk_overlap=100,
    length_function=len,
)
split_docs = text_splitter.split_documents(documents)

print(f"Split into {len(split_docs)} chunks for indexing")

print("Creating vector store...")

# 4. Create Vector Store with improved embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",  # Better embedding model
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

vectorstore = FAISS.from_documents(split_docs, embeddings)

print("Setting up language model...")

# 5. Setup LLM - using FLAN-T5-XL for better generation quality
model_id = "google/flan-t5-xl"  # Larger model for better output quality

# Use the correct model class for T5 (Seq2SeqLM instead of CausalLM)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

# Create text generation pipeline
text_generation_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
)

# Create LangChain pipeline
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

print("Creating improved RAG pipeline...")

# 6. Create Improved RAG Pipeline with hybrid search
retriever = vectorstore.as_retriever(
    search_type="mmr",  # Use Maximum Marginal Relevance to diversify results
    search_kwargs={"k": 8, "fetch_k": 20}  # Fetch more candidates, then select diverse subset
)

# Enhanced prompt template with better instructions
template = """
You are an expert restaurant information assistant. Your task is to provide detailed, accurate information about restaurants, menus, prices, and dining options based on the context provided.

CONTEXT INFORMATION:
{context}

USER QUESTION: {question}

INSTRUCTIONS:
1. Answer the question comprehensively based ONLY on the information provided in the context.
2. If comparing restaurants or menu items, provide specific details from the context to support your comparison.
3. For questions about menu items, include prices, descriptions, and any other relevant details.
4. When discussing vegetarian, dessert, or beverage options, list specific items that are available.
5. If discussing restaurant hours, be precise about the days and times provided in the context.
6. If the context doesn't contain enough information to fully answer the question, explain what information is available and what's missing.
7. DO NOT make up or infer information that isn't present in the context.
8. Format your answer in a clear, organized way with headers and bullet points when appropriate.

YOUR DETAILED ANSWER:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
)

def enhance_query(query):
    """Add relevant keywords to improve retrieval."""
    enhanced_query = query

    # Add relevant terms based on query content
    if "vegetarian" in query.lower() or "veg" in query.lower():
        enhanced_query += " vegetarian options menu items paneer"

    if "price" in query.lower() or "cost" in query.lower() or "expensive" in query.lower():
        enhanced_query += " price range cost rupees ₹"

    if "dessert" in query.lower() or "sweet" in query.lower():
        enhanced_query += " dessert sweet cake pastry truffle"

    if "hour" in query.lower() or "time" in query.lower() or "open" in query.lower():
        enhanced_query += " breakfast lunch dinner hours timing"

    if "compare" in query.lower():
        enhanced_query += " comparison differences similarities between locations"

    return enhanced_query

# Improved RAG chain with query enhancement
def rag_chain(query):
    # Enhance query for better retrieval
    enhanced_query = enhance_query(query)

    # Retrieve relevant documents
    docs = retriever.get_relevant_documents(enhanced_query)

    # Format context
    context_parts = []
    for i, doc in enumerate(docs, 1):
        context_parts.append(f"[Document {i}] {doc.page_content}")

    context = "\n\n".join(context_parts)

    # Create prompt
    formatted_prompt = prompt.format(context=context, question=query)

    # Generate response
    response = llm.invoke(formatted_prompt)
    return response

print("Setting up enhanced Gradio interface...")

# 7. Create Enhanced Gradio Interface
def process_query(query, history):
    if not query:
        return "Please ask a question about restaurants, menus, or food options."

    # Get response from RAG chain
    try:
        print(f"Processing query: {query}")
        response = rag_chain(query)
        print(f"Generated response: {response[:100]}...")  # Print preview of response
        return response
    except Exception as e:
        import traceback
        print(f"Error: {str(e)}")
        print(traceback.format_exc())
        return f"An error occurred while processing your question. Please try again with a different question."

# Interface title and description
title = "Premium Restaurant Information Assistant"
description = """
Get detailed information about high-end restaurants, their menus, pricing, and dining options. Ask me anything about:
- Restaurant hours, locations, and cuisine types
- Menu items and their descriptions
- Price ranges and value comparisons
- Vegetarian, dessert, and beverage options
- Comparisons between restaurant locations

Try asking specific questions to get the most helpful responses!
"""

# Create Gradio interface with better examples
demo = gr.ChatInterface(
    fn=process_query,
    title=title,
    description=description,
    examples=[
        "What are the breakfast, lunch and dinner hours for Capital Kitchen?",
        "List all vegetarian options available at Emperor Lounge with their prices",
        "Compare the cuisine types and operating hours of all Avartana locations",
        "What's the price range for pizzas at Capital Kitchen and what varieties do they offer?",
        "Tell me about dessert options available at Emperor Lounge",
        "What is the most expensive item on Capital Kitchen's menu?",
        "Are there any South Indian dishes available at Avartana and what are they?",
        "Compare the menu offerings between New Delhi, Mumbai and Kolkata locations"
    ],
    theme="soft"
)

print("Launching interface...")

# Launch the interface
demo.launch(share=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.5 MB/s eta 0:00:00

<ipython-input-1-a086de8b2b42>:178: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-1-a086de8b2b42>:208: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Creating improved RAG pipeline...
Setting up enhanced Gradio interface...
Launching interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2400bbfb285ea2daca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
